In [1]:
%%time
%load_ext autoreload
%autoreload 2

# Standard Python modules
import time 
import numpy as np
import os
import sys
import ipdb
import requests

sys.path.append(os.path.abspath(".."))
import json
import re
import glob
import asyncio
import aiohttp
import openai
from typing import Dict, Tuple, List, Callable
from dotenv import load_dotenv
from pathlib import Path
import PyPDF2

# Project imports
from src.utils import (timeit, encode_image, plt, pylab, find_bad_pagenos, 
                    delete_bad_pagenos, dump_fragmented_output_to_json, get_corresponding_bad_fnames,
                    load_output_from_json, logging_for_main, dump_output_to_json, log_execution_time, count_num_tokens)
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests_gpt import construct_payload_for_gpt, process_single_page
from src.api_requests_claude import construct_payload_for_claude
from src.document_generation import save_document, setup_logger, chapter_splitter

# Display and plotting, set notebook display width
from IPython.display import display, HTML, clear_output

display(HTML("<style>.container { width:90% !important; }</style>"))

# Print Python environment info
print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

# Load environment variables from .env file
env_path = Path('../.env')  # Adjust path if needed
load_dotenv(dotenv_path=env_path)

# OpenAI API setup
openai.api_key = os.getenv("OPENAI_API_KEY")

# Setup for PDF processing
foldername = "Der Weltkrieg v8"

# # split the pdf into separate pages
chapter_splitter(input_pdf_path=f"../input_data/{foldername} East Front.pdf", 
                 output_folder=f"../input_data/{foldername}")

chapter_splitter - L31 - INFO - Extracted 678 pages from PDF


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 



chapter_splitter - L45 - INFO - All pages have been split and saved to ../input_data/Der Weltkrieg v8


CPU times: user 1.59 s, sys: 2.25 s, total: 3.83 s
Wall time: 1.48 s


In [2]:
# Initialize variables
plotter = False
image_path = f"../input_data/{foldername}/*pdf"
fnames = sorted(glob.glob(image_path))
all_pagenos = [re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1) for fname in fnames]

# Storage for processed texts
raw_german_texts: Dict[str, str] = {}
german_texts: Dict[str, str] = {}
english_texts: Dict[str, str] = {}

# Configure logging
logger = setup_logger('time_logger')

---
## PART 1
## Fraktur Translator (GPT-4o)

In [3]:
# raw_german_texts, german_texts, english_texts, _ = load_output_from_json(foldername) 

In [4]:
@log_execution_time
async def main(fnames, model_name="", semaphore_count=10, extract=True):
    semaphore = asyncio.Semaphore(semaphore_count)  # Adjust number based on API limits
    async def wrapper_process_page(fname: str) -> Tuple[str, Dict]:
        global raw_german_texts, german_texts, english_texts  # Explicitly declare globals
        async with semaphore:
            pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
            result = await process_single_page(fname, model_name, plotter, pageno, extract) 
            return pageno, result
    
    # A list of coroutine objects. include only the unprocessed pages.
    tasks = []
    print(f'Composing tasks to be executed with asyncio, semaphore_count = {semaphore_count} ')
    for fname in fnames:
        pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
        if pageno not in set(raw_german_texts.keys()):
            print(f'{pageno} ', end='')
            tasks.append(wrapper_process_page(fname))
    print(f"main: len(tasks): {len(tasks)} -- Processing tasks as they complete")
    
    # Process tasks as they complete
    for i, task in enumerate(asyncio.as_completed(tasks)):
        try:
            pageno, (content, token_count, raw_german_text, german_text, english_text) = await task
            raw_german_texts[pageno] = raw_german_text
            german_texts[pageno] = german_text
            english_texts[pageno] = english_text
            
            logging_for_main(i, tasks, pageno, token_count, raw_german_text, german_text, english_text)
        except Exception as e:
            logger.error(f"{i} of {len(tasks)-1} - Error processing a task: {e}. token_count:{token_count}")
    return 

# Run the async code
await main(fnames, model_name="gpt-4o-2024-08-06", semaphore_count=10)


Composing tasks to be executed with asyncio, semaphore_count = 10 
001 002 003 004 005 006 007 008 009 010 011 012 013 014 015 016 017 018 019 020 021 022 023 024 025 026 027 028 029 030 031 032 033 034 035 036 037 038 039 040 041 042 043 044 045 046 047 048 049 050 051 052 053 054 055 056 057 058 059 060 061 062 063 064 065 066 067 068 069 070 071 072 073 074 075 076 077 078 079 080 081 082 083 084 085 086 087 088 089 090 091 092 093 094 095 096 097 098 099 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 2

logging_for_main - L259 - INFO - 0 of 677 -- Successfully processed pageno:437. token_count:2076
logging_for_main - L259 - INFO - 1 of 677 -- Successfully processed pageno:335. token_count:1975
logging_for_main - L259 - INFO - 2 of 677 -- Successfully processed pageno:333. token_count:2083
logging_for_main - L259 - INFO - 3 of 677 -- Successfully processed pageno:137. token_count:2175
logging_for_main - L259 - INFO - 4 of 677 -- Successfully processed pageno:520. token_count:2031
logging_for_main - L259 - INFO - 5 of 677 -- Successfully processed pageno:194. token_count:2040
logging_for_main - L259 - INFO - 6 of 677 -- Successfully processed pageno:660. token_count:3257
logging_for_main - L259 - INFO - 7 of 677 -- Successfully processed pageno:196. token_count:2112
logging_for_main - L259 - INFO - 8 of 677 -- Successfully processed pageno:659. token_count:3239
logging_for_main - L259 - INFO - 9 of 677 -- Successfully processed pageno:181. token_count:2041
logging_for_main - L259 - INFO

pageno:170[997]—Glinik line on May 16th and 17th. Simultaneously, the enemy, using relentless manpower, assaulted the bridgehead of Kolomea, whose now reinforced garrison held firm. However, the Russians succeeded on May 19th in breaking into the trenches on the southern Pruth bank at Kumaniczy, from which they were soon expelled. Then a longer pause in fighting occurred on the right flank of the 7th Army. General Freiherr von Pflanzer-Baltin refrained from resuming the offensive, considering the strength of the opposing enemy.</body>

------------------------------------------------------------------


logging_for_main - L259 - INFO - 195 of 677 -- Successfully processed pageno:616. token_count:1752
logging_for_main - L259 - INFO - 196 of 677 -- Successfully processed pageno:570. token_count:1715
logging_for_main - L259 - INFO - 197 of 677 -- Successfully processed pageno:647. token_count:2400
logging_for_main - L259 - INFO - 198 of 677 -- Successfully processed pageno:428. token_count:1881
logging_for_main - L259 - INFO - 199 of 677 -- Successfully processed pageno:427. token_count:2004
logging_for_main - L259 - INFO - 200 of 677 -- Successfully processed pageno:606. token_count:1859
logging_for_main - L259 - INFO - 201 of 677 -- Successfully processed pageno:265. token_count:2025
logging_for_main - L259 - INFO - 202 of 677 -- Successfully processed pageno:605. token_count:1212
extract_text_section - L243 - ERROR - pageno: 424, section name:"raw_german" was not found. token_count=5000
extract_text_section - L243 - ERROR - pageno: 424, section name:"german" was not found. token_count

pageno:511[of the 80th R. D.], 14th Ldw. D. and Abt. Etschel), Suter Group (XXI. A. K. [31st and 42nd S. D., 115th I. D., 77th R. D., Div. Zenker]), Eben Group (Gen. Kdo. I. A. K. with 2nd Ldw. D., 58th and 4th S. D. and 4th R. D.), Garnier Cavalry Corps (1st and 9th R. D., assigned 3rd R. D. of the Njemen Army). — 6th Ldw. Br. and Brig. Monteton had the strength of a division.
³) p. 535.</footer>

------------------------------------------------------------------


logging_for_main - L259 - INFO - 502 of 677 -- Successfully processed pageno:053. token_count:2012
logging_for_main - L259 - INFO - 503 of 677 -- Successfully processed pageno:244. token_count:2016
logging_for_main - L259 - INFO - 504 of 677 -- Successfully processed pageno:099. token_count:1893
logging_for_main - L259 - INFO - 505 of 677 -- Successfully processed pageno:517. token_count:2140
logging_for_main - L259 - INFO - 506 of 677 -- Successfully processed pageno:544. token_count:1830
logging_for_main - L259 - INFO - 507 of 677 -- Successfully processed pageno:100. token_count:1972
logging_for_main - L259 - INFO - 508 of 677 -- Successfully processed pageno:097. token_count:1904
logging_for_main - L259 - INFO - 509 of 677 -- Successfully processed pageno:536. token_count:1654
logging_for_main - L259 - INFO - 510 of 677 -- Successfully processed pageno:342. token_count:1941
logging_for_main - L259 - INFO - 511 of 677 -- Successfully processed pageno:359. token_count:1946
logging_fo

pageno:559[Sommer]) and 17th Ldw. D. (Abt. Eisebed) were unnamed or newly formed.
</footer>

------------------------------------------------------------------


logging_for_main - L259 - INFO - 545 of 677 -- Successfully processed pageno:494. token_count:2028
logging_for_main - L259 - INFO - 546 of 677 -- Successfully processed pageno:114. token_count:1083
logging_for_main - L259 - INFO - 547 of 677 -- Successfully processed pageno:549. token_count:1924
logging_for_main - L259 - INFO - 548 of 677 -- Successfully processed pageno:020. token_count:1830
logging_for_main - L259 - INFO - 549 of 677 -- Successfully processed pageno:125. token_count:1974
logging_for_main - L259 - INFO - 550 of 677 -- Successfully processed pageno:316. token_count:1812
logging_for_main - L259 - INFO - 551 of 677 -- Successfully processed pageno:540. token_count:2015
logging_for_main - L259 - INFO - 552 of 677 -- Successfully processed pageno:021. token_count:1883
logging_for_main - L259 - INFO - 553 of 677 -- Successfully processed pageno:330. token_count:2065
logging_for_main - L259 - INFO - 554 of 677 -- Successfully processed pageno:360. token_count:1880
logging_fo

In [14]:
## V5
good_pagenos = {}
good_pagenos['Der Weltkrieg v5'] = {'213', '225', '221'}
good_pagenos['Der Weltkrieg v6'] = {'377', '378', '379'}

## V7 ('417' and '429' does not get OCRed by GPT-4o, and it gets summarized by Claude)
good_pagenos['Der Weltkrieg v7'] = {'265', '407', '409', '410', '412', '418', '420', '421', '423', '424', '428'} 

## v10
good_pagenos['Der Weltkrieg v10'] = {'083'}
good_pagenos['Der Weltkrieg v11'] = {}
good_pagenos['Der Weltkrieg v2'] = {'246','247'}

## V8 
good_pagenos['Der Weltkrieg v8'] = {'170', '511', '559', '678'}

bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])
# delete_bad_pagenos(bad_pagenos, raw_german_texts, german_texts, english_texts)


> pageno: 004. `raw_german_texts` pageno: 004, "<raw_german>" section was not found

> pageno: 006. `raw_german_texts` pageno: 006, "<raw_german>" section was not found

> pageno: 007. `raw_german_texts` pageno: 007, "<raw_german>" section was not found

> pageno: 008. `raw_german_texts` pageno: 008, "<raw_german>" section was not found

> pageno: 014. `raw_german_texts` pageno: 014, "<raw_german>" section was not found

> pageno: 334. `raw_german_texts` pageno: 334, "<raw_german>" section was not found

> pageno: 424. `raw_german_texts` pageno: 424, "<raw_german>" section was not found

> pageno: 630. `raw_german_texts` pageno: 630, "<raw_german>" section was not found

> pageno: 651. `english_texts` pageno: 651, "<english>" section was not found

> pageno: 656. `english_texts` pageno: 656, "<english>" section was not found

> pageno: 670. `raw_german_texts` pageno: 670, "<raw_german>" section was not found

> pageno: 678. `raw_german_texts` pageno: 678, "<raw_german>" section was no

---
## PART 2
##  Handle missing keys (Claude Sonnet)

In [51]:
# Check for unprocessed pages and print them to stdout.
# ------------------------------------------------------------------------------------
# 1. Rerun the missing pages on Claude without performing FFT based extraction. 
# ------------------------------------------------------------------------------------

# Get missing keys based on empty raw_german_texts
bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])
delete_bad_pagenos(bad_pagenos, raw_german_texts, german_texts, english_texts)
await main(fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=False)

# ------------------------------------------------------------------------------------
# 2. If there still are missing pages, run them performing FFT based extraction. 
#    This time compute missing_keys based on 'english_texts'.
# ------------------------------------------------------------------------------------

# Now get missing keys based on empty english_texts
bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])
delete_bad_pagenos(bad_pagenos, raw_german_texts, german_texts, english_texts)
await main(fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=True)

bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])
delete_bad_pagenos(bad_pagenos, raw_german_texts, german_texts, english_texts)
await main(fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=False)

bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])



> pageno: 651. `english_texts` [Continued translation of all entries maintaining the same format and structure as the original, including all page numbers and military ranks/positions]
</body>


> pageno: 656. `raw_german_texts` [continued transcription of all entries through to "von Mackenſen"]


> pageno: 670. `raw_german_texts` [Content continues with detailed listings for 3. Armee through Gruppe Mittel]
Weltkrieg. VIII. Band.                           43


bad_pagenos (3): ['651', '656', '670']
------------------------------------------------------------------
> Deleting pageno: 651. Please run `await main(fnames, model_name, semaphore_count)` again 
> Deleting pageno: 656. Please run `await main(fnames, model_name, semaphore_count)` again 
> Deleting pageno: 670. Please run `await main(fnames, model_name, semaphore_count)` again 
Composing tasks to be executed with asyncio, semaphore_count = 1 
651 656 670 main: len(tasks): 3 -- Processing tasks as they complete


logging_for_main - L252 - ERROR - main pageno:651. '[' present in english_text. token_count:1600


pageno:651[Continuing with faithful translations of all entries, maintaining the same structure and reference numbers as in the original]
</body>

------------------------------------------------------------------


extract_text_section - L243 - ERROR - pageno: 670, section name:"raw_german" was not found. token_count=714
logging_for_main - L252 - ERROR - main pageno:670. '[' present in english_text. token_count:714


pageno:670[Content continues with detailed listings for 3rd Army through Group Mittel]
World War. Volume VIII.
</body>
<pageno>43</pageno>

------------------------------------------------------------------


...

pageno:651[Continuing with faithful translations of all entries, maintaining all military ranks, unit designations, and page numbers as in the original]
</body>

------------------------------------------------------------------


logging_for_main - L252 - ERROR - main pageno:670. '[' present in english_text. token_count:1492


pageno:670[Content continues with same format through all army listings and corps]</body>
<footer>World War. Volume VIII.                43</footer>

------------------------------------------------------------------


logging_for_main - L252 - ERROR - main pageno:656. '[' present in english_text. token_count:1734
wrapper - L88 - INFO - Finished main in 100.34 seconds.


pageno:656[Continuing with all entries in the same detailed manner, maintaining all military ranks, unit designations, and page references as in the original]
</body>

------------------------------------------------------------------

> pageno: 651. `english_texts` [Continuing with faithful translations of all entries, maintaining all military ranks, unit designations, and page numbers as in the original]
</body>


> pageno: 656. `english_texts` [Continuing with all entries in the same detailed manner, maintaining all military ranks, unit designations, and page references as in the original]
</body>


> pageno: 670. `raw_german_texts` [Continued content follows same format through XIX. Korps and various other Korps and Gruppe listings]
Weltkrieg. VIII. Band.                43


bad_pagenos (3): ['651', '656', '670']


In [61]:
# save json outputs
dump_output_to_json(foldername, raw_german_texts, german_texts, english_texts)

# Save json files and .docx files.
doc1, fname1 = save_document(german_texts, foldername, language=f'{foldername} - German')
doc2, fname2 = save_document(english_texts, foldername, language=f'{foldername} - English')



dumped to ../output_data/Der Weltkrieg v8/*json files.


save_document - L197 - INFO - saved to "../output_data/Der Weltkrieg v8/Der Weltkrieg v8 - German"
save_document - L197 - INFO - saved to "../output_data/Der Weltkrieg v8/Der Weltkrieg v8 - English"


``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

```

### Available models and pricing:
```
"gpt-4o-2024-08-06":
    "price_txt": "$2.50 / 1M input tokens"
    "price_img": "$0.001913 / 1500px^2"
    
"gpt-4o-mini-2024-07-18":
    "price_txt": "$0.150 / 1M input tokens"
    "price_img": "$0.003825 / 1500px^2"
    
```

---
## PART 3
## Load the German text and translate fragmented sentences.

In [62]:
# initialize output dicts
english_texts_defragmented = {}
contents = {}
payloads = {}
fragments_2 = {None:'', all_pagenos[-1]:''}

# raw_german_texts, german_texts, english_texts, english_texts_defragmented = load_output_from_json(foldername, load_defrag=True)

In [211]:
from src.utils import log_execution_time_synchronous
from src.api_requests_gpt import make_gpt_request_for_broken_sentences, construct_gpt_payload_fragmented_sentences
from src.api_requests_claude import make_claude_request_for_broken_sentences, construct_claude_payload_fragmented_sentences


@log_execution_time_synchronous
def main_broken_sentences(model_name):
    global payloads, contents, fragments_2
    for i in range(len(all_pagenos) - 1):
        pageno = all_pagenos[i]
        prev_pageno = all_pagenos[i-1] if i > 0 else None
        next_pageno = all_pagenos[i+1]
        if pageno in english_texts_defragmented.keys() and len(english_texts_defragmented[pageno]) > 10:
            logger.info(f'pageno:{pageno} already in `english_texts_defragmented`... skipping')
            continue
        
        for trial in [1,2,3]:
            try:
                logger.info(f"Processing pageno: {pageno} {' ...trying again '+ str(trial) if trial > 1 else ''}")
                time.sleep(.1)
                if model_name.startswith('gpt'):
                    construct_payload = construct_gpt_payload_fragmented_sentences
                    make_llm_request = make_gpt_request_for_broken_sentences
                else:
                    construct_payload = construct_claude_payload_fragmented_sentences
                    make_llm_request = make_claude_request_for_broken_sentences

                payload = construct_payload(
                        german_page_1=german_texts[pageno],
                        german_page_2=german_texts[next_pageno],
                        english_page_1_old_input=english_texts[pageno],
                        german_page_1_top_fragment_to_be_ignored=fragments_2[prev_pageno] # this is in German 
                )
                payloads[pageno] = payload['messages'][1]['content'][0]['text']
                content = make_llm_request(payload, pageno, english_texts_defragmented)
                contents[pageno] = content
                fragments_2[pageno] = re.search(r'<fragment_2>(.*?)</fragment_2>', content, re.DOTALL).group(1)
                if fragments_2[pageno].count('\n') > 10:
                    print('not accepting fragment_2. fragments_2[pageno].count("\\n")', fragments_2[pageno].count('\n'))
                    fragments_2[pageno] = '' 
                logger.info(f"pageno:{pageno} next round's `german_page_1_top_fragment_to_be_ignored`: {fragments_2[pageno]}") 
                break
            except Exception as e:
                logger.error(f"Error processing. {e}. trial:{trial}, pageno: {pageno}")
                if trial == 3:
                    english_texts_defragmented[pageno] = ''

        if pageno[-1] == '0' or pageno == all_pagenos[-2]:
            dump_fragmented_output_to_json(foldername, english_texts_defragmented, fragments_2, contents)

    english_texts_defragmented[next_pageno] = english_texts[next_pageno]

main_broken_sentences(model_name='gpt') 

In [227]:
bad_pagenos = []
for pageno in all_pagenos:
    if '<body>' not in english_texts_defragmented[pageno] and '</body>' not in english_texts_defragmented[pageno]:
        bad_pagenos.append(pageno)
print(f"{bad_pagenos}: ({len(bad_pagenos)})")

for pageno in bad_pagenos:
    del english_texts_defragmented[pageno]
    
main_broken_sentences(model_name='gpt') 


[]: (0)


In [213]:
# if no fragmented pages then copy english_texts -> english_texts_defragmented.

for pageno in all_pagenos:
    if not fragments_2[pageno]:
        english_texts_defragmented[pageno] = english_texts[pageno] 

In [229]:
dump_fragmented_output_to_json(foldername, english_texts_defragmented, fragments_2, contents)

doc1, fname1 = save_document(german_texts, foldername, language=f'{foldername} - German_defragmented')
doc2, fname2 = save_document(english_texts_defragmented, foldername, language=f'{foldername} - English_defragmented')


dumped fragmented outputs foldername: Der Weltkrieg v8.


save_document - L197 - INFO - saved to "../output_data/Der Weltkrieg v8/Der Weltkrieg v8 - German_defragmented"
save_document - L197 - INFO - saved to "../output_data/Der Weltkrieg v8/Der Weltkrieg v8 - English_defragmented"


---

## PART 4
## Fix Broken Paragraphs

In [230]:
from src.constants import *
# print(CARRIAGE_RETURN_SYSTEM_PROMPT)
# print()
# print(CARRIAGE_RETURN_USER_PROMPT)

def construct_payload_for_gpt_slash_n(english_text_defragmented) -> dict:
    model_name = "gpt-4o-2024-08-06"
    payload = {
        "model": model_name,
        "messages": [
          {
            "role": "system", 
            "content": CARRIAGE_RETURN_SYSTEM_PROMPT
          },
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": CARRIAGE_RETURN_USER_PROMPT.format(
                    english_text_defragmented=english_text_defragmented
                )},
            ]
          }
        ],
        "max_tokens": 5000,
        "temperature": 0.1
    }
    return payload


In [231]:
problematic = {}
for pageno in all_pagenos:
    text = english_texts_defragmented[pageno]
    
    n_count = text.count('\n')
    if n_count > 15:
        problematic[pageno] = text

print('len(problematic)', len(problematic))

for pageno in problematic:
    print('pageno:', pageno)
    print(problematic[pageno])

len(problematic) 161
pageno: 004

<header>Table of Contents.
The Operations of the Year 1915.
The Events in the West in Spring and Summer,
in the East from Spring until the End of the Year.</header>
<body>                                                                Page
I. The Situation of the Central Powers in May 1915 . . . . . . 1
    1. Italy's Entry into the War . . . . . . . . . . . . . . . . 1
    2. The Worsening of the Economic Situation of the
       Central Powers and the Submarine Trade
       War . . . . . . . . . . . . . . . . . . . . . . . . . 12
    3. The Manpower Replacement and Ammunition Situation until
       the End of 1915 . . . . . . . . . . . . . . . . . . . . . 18
II. The Deployment and First Battles on the Italian
    Front . . . . . . . . . . . . . . . . . . . . . . . . 25
III. The Western Front from Mid-April to Early
    August 1915 . . . . . . . . . . . . . . . . . . . . . . . . 34
    1. The Battles until the Beginning of the Spring
       Offensive i

In [232]:
async def make_gpt_request_slash_n(text) -> dict:
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(
            "https://api.openai.com/v1/chat/completions",
            json=construct_payload_for_gpt_slash_n(text),
            headers=headers
        ) as response:
            return await response.json()


In [ ]:
@log_execution_time
async def main_slash_n(model_name="", semaphore_count=10):
    semaphore = asyncio.Semaphore(semaphore_count) 
    async def wrapper_process_page(problematic, pageno: str) -> Tuple[str, Dict]:
        async with semaphore:
            response = await make_gpt_request_slash_n(problematic[pageno]) 
            return pageno, response

    tasks = []
    print(f'Composing tasks to be executed with asyncio, semaphore_count = {semaphore_count}')
    for q, pageno in enumerate(problematic):
        print(f'{pageno} ', end='')
        tasks.append(wrapper_process_page(problematic, pageno))
    print(f"main: len(tasks): {len(tasks)} -- Processing tasks as they complete")

    responses = []
    for i, task in enumerate(asyncio.as_completed(tasks)):
        try:
            pageno, response = await task
            responses.append((pageno, response))
        except Exception as e:
            logger.error(f"{i} of {len(tasks)-1} - Error processing a task: {e}.")
    return responses

# Run the async code
responses = await main_slash_n(model_name="gpt-4o-2024-08-06")


Composing tasks to be executed with asyncio, semaphore_count = 10
004 005 006 007 008 009 010 011 013 014 020 022 025 029 032 033 035 038 044 048 052 060 073 082 083 086 088 089 091 095 097 098 106 116 123 126 128 131 132 135 143 147 152 155 158 159 162 174 176 178 192 193 203 210 218 220 221 224 225 226 237 239 240 242 248 262 268 274 275 282 286 293 295 323 330 349 353 354 357 359 367 375 380 385 394 398 399 402 403 407 419 423 425 434 438 444 449 454 457 458 469 470 471 472 473 476 477 483 484 486 490 493 503 505 509 517 521 525 531 536 542 547 573 579 585 587 625 627 639 641 644 645 646 647 648 649 650 652 653 654 655 657 658 659 660 661 662 663 664 665 666 667 668 669 671 672 673 674 675 676 677 main: len(tasks): 161 -- Processing tasks as they complete


In [16]:
good_pagenos = {}
good_pagenos['Der Weltkrieg v8'] = {''}
good_pagenos['Der Weltkrieg v7'] = {''}
good_pagenos['Der Weltkrieg v2'] = {''}


In [17]:
from IPython.display import display, HTML
english_texts_fixed_paragraphs = {key: value for key, value in english_texts_defragmented.items()}

for pageno, response in sorted(responses):
    if pageno in good_pagenos[foldername]:
        continue
    result = response['choices'][0]['message']['content']
    match = re.search(r'<output_text>(.*?)</output_text>', result, re.DOTALL)
    if match:
        result = match.group(1).strip()  # Extract content and strip any leading/trailing whitespace

    print('---' * 22)
    print(f"pageno: {pageno}")
    if (len(result) < 40 and 'The input text is' in result and 'correct' in result):
        print(f'result: {result}')
    else:
        print(f'\n{english_texts_defragmented[pageno]}')
        print('result: ')
        display(HTML(f'<b style="color:blue;">{result}</b>'))
        english_texts_fixed_paragraphs[pageno] = result


------------------------------------------------------------------
pageno: 005
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 006
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 007
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 008
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 013


<pageno>6</pageno>
<header>The Agreements with Austria-Hungary.</header>
<body>that Serbia will behave calmly seems to me quite likely." However, if "despite everything" a great war comes,
Germany would be forced to first strike against France,
against which General v. Moltke, like Count Schlieffen, considered a
quick decision possible. He stated that through such a
decision, effective assistance would ultimately be provide

------------------------------------------------------------------
pageno: 014

to cooperate, then "purely spatially theoretically taken" could even 
be in question, to move back the Austro-Hungarian deployment under occupation 
of the Carpathian passes to the line Bartfeld—Neu-Sandez—Bochnia 
(35 km east of Krakow) — that meant far into West Galicia. However, there are great concerns about this. On the other hand, a deployment 
in the line Rudki (40 km southwest of Lemberg)—San estuary might be advantageous, 
to proceed from there to the offensive as soon as the Russian 
forces have reached the area of impact. General v. Conrad ties 
the question of the strength and behavior of the German forces remaining in the east 
in a two-front war, and whether they would be able to "bind 19½ Russian divisions." 
Furthermore, he asked about the strength, arrival time, and use of the reinforcements 
expected later from the western theater of war. 
General v. Moltke replied on February 24, 1909, th

------------------------------------------------------------------
pageno: 019


<pageno>12</pageno>
<header>The Agreements with Austria-Hungary.</header>
<body>Forces carry more than ever a danger within them, and continued: "If 
Austria needs all its forces to carry out the fight against Russia, the same 
applies to Germany in the fight against France. I would therefore advocate 
deploying our troops prepared in the East in the West as well, if 
consideration for Austria did not prevent me from doing so. In the 
resolution of the conflict between Germany and France lies, in my 
conviction, the focal point of the entire European war, and" (as stated in 
the Schlieffen memorandum) "the fate of Austria will not be decided on 
the Bug, but on the Seine."
Furthermore, General v. Moltke sought to accommodate the wishes of 
General v. Conrad as much as possible by expressing the hope that it might 
very soon be possible, after the opening of hostilities in the West, to deploy 
additional se

------------------------------------------------------------------
pageno: 020


<pageno>13</pageno>
<header>Moltke's View on the Role of Austria-Hungary.</header>
<body>
The command against Russia, which might become desirable or even necessary,
was dispensable for the first weeks of the war. The extremely delicate
and hardly satisfactorily solvable question of the supreme command in the East
was therefore not addressed in the peace negotiations between the two chiefs of staff.
As far as the position of the allied powers towards each other
seemed possible, the cooperation of the German and Austro-Hungarian
forces for the case of war was regulated. That nevertheless
differences of opinion remained in the basic views
is not surprising given the nature of the alliance relationship:
Thus, General v. Conrad never completely abandoned the idea
of possibly waging a war in the Balkans, regardless
of the considerable weakening of the forces available against Russia
and the resulting additional

------------------------------------------------------------------
pageno: 023


<pageno>16</pageno>
<header>The Armaments and Force Ratios until Summer 1914.</header>
<body>raises several points about the war readiness of the other great powers including Germany, namely: through the elimination of the military weakness period in winter due to retention of the oldest year group until the completed training of recruits, — through frequent practical testing of all mobilization measures with the help of trial and test mobilizations, — through the possibility of extraordinary acceleration of mobilization with the help of the "war preparation period". Based on intelligence about measures during the Balkan Wars of 1912/13, the memorandum further explained that this "war preparation period" allowed, in times of political tension, "to assign supplementary personnel and horses to the troops before the start of actual mobilization" and to prepare these units to such an extent that they could ult

...

------------------------------------------------------------------
pageno: 348


<header>Appendix 1.</header>
<header>Military Formations.</header>
<header>Preliminary Note.</header>
<body>
The military formations reflect the composition of the armies for a specific point in time or period; earlier or later deviations in the composition are indicated. All parts added later are highlighted by special script.
The following abbreviations, which may not be immediately understandable, were used:
A.K. = Army Corps
Abt. = Division
Drag.R. = Dragoon Regiment
Erg./147 = Replacement Battalion Infantry Regiment No. 147
F.A.R. = Field Artillery Regiment
Füs.R. = Fusilier Regiment
G.R. = Guard Regiment
Geb.Brig. = Mountain Brigade
Gr.R. = Grenadier Regiment
Hus.R. = Hussar Regiment
Inf.R. = Infantry Regiment
Jäg.B. = Jäger Battalion
Jäg.R. z. Pf. = Mounted Jäger Regiment
K.R. = Cuirassier Regiment
Ldst.R. = Landsturm
Ldsch. = Landwehr
M.G.A. = Machine Gun Division
Pi.1 = Pioneer Battalion No. 1
1./

------------------------------------------------------------------
pageno: 349


<pageno>358</pageno>
<header>Order of Battle.</header>
<body>
The German 8th Army
on August 26, 1914, 1st day of the Battle of Tannenberg¹).
9 Inf. and Res. Div., 4 Div. Det. and Replacement Troops, 1 Cav. Div. = 158 Batl., 78 Squadrons
140 Batteries (= 774 Guns²).
Strengthened by early September to:
13 Inf. and Res. Div., 5½ Div. Landwehr and Replacement Troops, 2 Cav. Div. = 229 Batl., 119 Squadrons
219 Batteries (= 1194 Guns).
Additionally, Landwehr Corps with 2 Landwehr Div. = 34 Batl., 12 Squadrons, 12 Batteries
(= 72 Guns).
Army High Command 8.
Commander-in-Chief: Generaloberst v. Beneckendorff and v. Hindenburg (until 22. 8. General-
oberst v. Prittwitz and Gaffron)
Chief of Gen. Staff: Genmaj. Ludendorff (until 22. 8. Genmaj. Count v. Waldersee)
1st Gen. Staff Officer: Lt. Col. Hoffmann
Quartermaster General: Genmaj. Grünert
Gen. of Engineers: Genmaj. Resten.
Air Forces.
Field Flying Det. 16
Fortre

------------------------------------------------------------------
pageno: 350
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 351
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 352
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 353


<pageno>362</pageno>
<header>Military Formations.</header>
<body>
Fort Boyen (Lötzen).
4½ Battalions, 1 Squadron, 8 Batteries.
Commander: Colonel Busse
Infantry: Replacement/R.R. 147, IV./Ldw. 18 and 2½ Battalions Ldst.
Machine Guns: 11 Fortress Detachments and Troops
Cavalry: Replacement Squadron Drag. R. 11
Field Artillery: 2 Ldw. Batteries from XX. A.K.
Foot Artillery: 6 unassigned Batteries.
Pioneers: 1 Fortress Battalion Detachment.
Air Forces: 1 XX. Army Corps.
From the Army's Line of Communication Inspection (on 30th and 31st August
call

------------------------------------------------------------------
pageno: 354
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 355
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 356
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 357
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 358


<pageno>367</pageno>
<header>The Austro-Hungarian Army.</header>
<body>
2nd Army.
7 Inf. Div., 4 Lfst. Brig., 2 Cav. Div.
= 145 1/4 Batt., 67 3/4 Squadrons, 64 Batteries (= 368 Guns)
Part of the army formed the Kövess Army Group until August 23 (see below).
Army Commander: General of Cavalry v. Böhm-Ermolli ¹)
Chief of Gen. Staff: Genmaj. v. Meenfessy
IV Corps ¹): 31 ¹) and 32 ¹) Inf. Div., 4th March Brig.
VII Corps ¹): 17 

------------------------------------------------------------------
pageno: 359
result: "The input text is correctly OCR'ed."
------------------------------------------------------------------
pageno: 360
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 361
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 362
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 363
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 364
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 365
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 366
result: "The input text is correctly OCR'ed."
------------

In [18]:
from src.utils import dump_fixed_paragraphs_output_to_json

dump_fixed_paragraphs_output_to_json(foldername, english_texts_fixed_paragraphs)

doc2, fname2 = save_document(english_texts_fixed_paragraphs, foldername, language=f'{foldername} - English_fixed_paragraphs')


save_document - L197 - INFO - saved to "../output_data/Der Weltkrieg v2/Der Weltkrieg v2 - English_fixed_paragraphs"


In [50]:
# print(german_texts['017'])
print(english_texts_defragmented['017'])
print(english_texts_fixed_paragraphs['017']) 



<pageno>10</pageno>
<header>The Agreements with Austria-Hungary.</header>
<body>
Support was expected. However, given the circumstances, a decision against France was impossible. After all, the Bosnian crisis had just reached its peak. In such a situation, one could not question the results of all previous negotiations but had to try to help as best as possible. Thus, General v. Moltke — albeit with a heavy heart — decided to make the commitment.
He wrote about it: An attack by weak German forces against the fortified Narew line would indeed face great difficulties and was also "threatened on the right flank by Warsaw, on the left by counterattacks from Lomsha. — Nevertheless, I will not hesitate to make the attack to support the simultaneous Austrian offensive. Your Excellencies can rely on this promise, which is well considered. The condition is that the movements of the allies are initiated simultaneously and carried out unconditionally. — Should the execution of the intentions be 

In [16]:
# print(raw_german_texts['117'])
# print(german_texts['117'])
print(english_texts['117'])
print(english_texts_defragmented['117'])


<pageno>211</pageno>
<header>The Southern Army Halts the Advance to the Southeast.</header>
<body>General von Linsingen, however, did not consider the movements of the troops under Field Marshal Lieutenant Szurmay sufficient to reliably secure his left flank. This seemed assured to him only if the enemy was pushed back over the Dniester, which could only be achieved by deploying stronger forces. Reserves were not available. The divisions of Bothmer's corps stationed on the northern bank of the Dniester were also engaged in fierce defensive battles since early morning and urgently needed support themselves. Under these circumstances, there was no choice but to refrain from continuing the offensive of the right wing of the army in a northeasterly and northerly direction and to focus on defending what had been achieved, in order to free up his main forces for a counterattack to the northwest. On the morning of June 9, General von Linsingen ordered the Gerok corps, which was engaged in an